# **Proyecto Instagram**

# Cuenta uno:
### **Email:**

redinstagramapp@gmail.com

redinstagramapp2022


### **Instagram:**

@proyectoredargentina

redinstagram2019


# Cuenta dos:
### **Email:**

redinstagramapp2@gmail.com

redinstagramapp2022


### **Instagram:**

@proyectoredarg

redinstagram2021

## **Facebook:**
redinstagramapp@gmail.com

redinstagramapp2022

In [1]:
# Usuarios a Scrapear
usuarios_interes = [
    
'mauriciomacri', 'cristinafkirchner', 'horaciorodriguezlarreta', 
 'joseluisespert', 'silbatakis', 'kicillofok', 'elisalilitacarrio','sergiomassaok', 
'coqui.capitanich', 'aliciakirchner', 'sergiounac', 'gustavovaldesok'
'juanluismanzur', 'gerardomoralesjujuy', 'gzamorasde', 'jschiaretti'         
'mariuvidal','patobullrich','gugalusto','alferdezok','javiermilei',  
    
]

In [2]:
# Selenium imports
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

#Other imports
import os
from time import sleep
import random
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook
import re

In [3]:
# Usuarios a Scrapear
df = pd.read_excel('//senadores.xlsx')
df.fillna('', inplace=True)
df = df[df['igs']!='']
usuarios_interes = [u.split('/')[3] for u in df['igs'].values]
usuarios_interes = usuarios_interes[usuarios_interes.index('maggiesolariq'):]

In [4]:
# Credenciales
user = "proyectoredargentina"
password = "redinstagram2019"

In [5]:
class igScraper():
    def __init__(self):
        pass

    def login(self, user, password, navegacion_con_headers = False):

        # Funcion para evadir carteles
        def clickear(texto):
            self.path = f'//*[text() = "{texto}"]'
            self.boton = self.driver.find_element('xpath', self.path) 
            sleep(1)
            self.boton.click()
            sleep(2)

        # instanciamos webdriver (True or False)
        if navegacion_con_headers == True:
            self.driver = webdriver.Chrome(ChromeDriverManager().install())
        else:
            self.driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

        # ir a instagram.com
        self.driver.get('https://instagram.com/')
        sleep(10)

        # poner usuario
        username = self.driver.find_element('name','username')
        sleep(1)
        username.click()
        sleep(1)
        username.send_keys(user)
        sleep(1)

        # poner contraseña
        pw = self.driver.find_element('name','password')
        sleep(2)
        pw.click()
        sleep(2)
        pw.send_keys(password)
        sleep(2)

        # login
        try:
            login_btn = self.driver.find_element('xpath','//div[text() = "Iniciar sesión"]')
        except:
            login_btn = self.driver.find_element('xpath','//div[text() = "Log In"]')
        sleep(2)
        login_btn.click()
        sleep(4)


        # Apretar "ahora no" a los pop ups, hasta que pueda buscarse un usuario
        contador_intentos = 0
        carteles_sorteados = False
        while carteles_sorteados == False and contador_intentos < 200:
            contador_intentos+=1
            try:
                buscador = self.driver.find_element('xpath',"//input[@placeholder='Buscar']")
                carteles_sorteados = True
            except:
                try:
                    buscador = self.driver.find_element('xpath',"//input[@placeholder='Search']")
                    carteles_sorteados = True
                except:
                    try:
                        clickear('Not Now')      
                    except:
                        try:
                            clickear('Ahora no')
                        except:
                            pass

        if contador_intentos == 200:
            print('\n\nEs posible que la cuenta haya sido blockeada.\n\n')

    def descargar(self, usuario, listado_seguidores, nombre):
        print(f'Guardando {nombre} de {usuario}...\n\n')
        nombre += '.xlsx'
        sumar = 0
        try:
            wb_obj = openpyxl.load_workbook(nombre)
            sumar += 1
        except:
            Workbook().save(nombre)
            wb_obj = openpyxl.load_workbook(nombre)

        # sheet
        worksheet = wb_obj.worksheets[0]

        # Columna de este usuario
        columna = worksheet.max_column + sumar

        # Nombre columna (usuario)
        worksheet.cell(row = 1, column = columna).value = usuario

        # Debajo, sus seguidos/seguidoress
        for i,s in enumerate(listado_seguidores):
            worksheet.cell(row = i+2, column = columna).value = s

        wb_obj.save(nombre)
        wb_obj.close()


    # SEGUIDORES
    def get_seguidores(self, usuario, cantidad_usuarios_a_scrapear= float('inf'), download= True):

        print(f'Scrapeando seguidores de {usuario}\n\n')

        url_usuario = 'https://www.instagram.com/'+usuario

        # Entrar al usuario
        self.driver.get(url_usuario)
        sleep(8)

        # Acceder a seguidores
        seguidores_boton = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[2]//span")
        seguidores_boton.click()
        sleep(3)

        # cantidad de seguidores
        can_seguidores = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[2]//span").text
        if '.' in can_seguidores:
            cantidad_decimal = len(can_seguidores.split('.')[-1][:-1])
        else:
            cantidad_decimal = 0
        can_seguidores = int(can_seguidores.replace('K','0'*(3-cantidad_decimal)).replace('M','0'*(6-cantidad_decimal)).replace('.','').replace(',',''))
        self.can_seguidores = can_seguidores
        
        # cuanto hay que scrollear
        scrollear_inicial = round(can_seguidores/12)+1

        # instanciamos seguidores_total
        seguidores_total = []
        cantidad_encontrados = []

        # scrollear y apendear
        for i in range(scrollear_inicial):
            i += 1

            #Cada x scrolls, pausa de x minutos
            if i % 10 == 0:
                minutos = random.uniform(3, 7)
                print(f'\n\n\nHaciendo pausa de {round(minutos)} minutos\n\n\n')
                time.sleep(minutos*60)

            # Cada x scrolls, scroll up to confuse bot detectors
            if i % 15 == 0:    
                self.driver.execute_script("""
                    var scrollt = document.querySelector('div[role="dialog"] ._aano')
                    scrollt.scrollTop = scrollt.scrollHeight - (scrollt.scrollHeight / 5)
                    """)

            # Print
            print(f'Scrolleando {i} de {scrollear_inicial}')

            # Scroll down
            self.driver.execute_script("""
                var scrollt = document.querySelector('div[role="dialog"] ._aano')
                scrollt.scrollTop = scrollt.scrollHeight
                """)

            time.sleep(random.uniform(20, 25))

            # Nuevos seguidores
            # Español
            seguidores = self.driver.find_elements('xpath',f"(//div[@class='_aae-']/li)[position()>={len(seguidores_total)}]//*/a")
            seguidores = [u.text for u in seguidores if len(u.text)>0]
            if len(seguidores) == 0:
                #Ingles
                seguidores = self.driver.find_elements('xpath',f"//div[position()>={len(seguidores_total)}]//*/span/a[@role = 'link']")
                seguidores = [elem.get_attribute('href').split('/')[-2] for elem in seguidores]

            # agregamos los nuevos seguidores
            seguidores_total.extend(seguidores)
            print(f'Hasta el momento, se han encontrado {len(seguidores_total)} seguidores de {usuario}\n')

            # Si no encontro nada en las ultimas 3 iters, break
            cantidad_encontrados.append(len(seguidores_total))
            try:
                ultimos = 20
                if len(set(cantidad_encontrados[-ultimos:])) == 1 and len(cantidad_encontrados) > ultimos:
                    print('\n\n\nINTERRUPCION: NO SE ENCUENTRAN NUEVAS CUENTAS\n')
                    break
            except:
                pass

            # break si se alcanzo maximo de usuarios
            if len(seguidores_total) > cantidad_usuarios_a_scrapear:
                break

            time.sleep(random.uniform(20, 25))

        return seguidores_total

    # SEGUIDOS
    def get_seguidos(self, usuario, cantidad_usuarios_a_scrapear= float('inf'), download= True):

        print(f'Scrapeando seguidos de {usuario}\n\n')

        url_usuario = 'https://www.instagram.com/'+usuario

        # Entrar al usuario
        self.driver.get(url_usuario)
        sleep(8)

        # Acceder a seguidos
        seguidos_boton = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[3]//span")
        seguidos_boton.click()
        sleep(3)

        # cantidad de seguidos
        can_seguidos = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[3]//span").text
        if '.' in can_seguidos:
            cantidad_decimal = len(can_seguidos.split('.')[-1][:-1])
        else:
            cantidad_decimal = 0
        can_seguidos = int(can_seguidos.replace('K','0'*(3-cantidad_decimal)).replace('M','0'*(6-cantidad_decimal)).replace('.','').replace(',',''))
        self.can_seguidos = can_seguidos
        
        # cuanto hay que scrollear
        scrollear_inicial = round(can_seguidos/12)+1

        # instanciamos seguidos_total
        seguidos_total = []
        cantidad_encontrados = []

        # scrollear y apendear
        for i in range(scrollear_inicial):
            i += 1

            # Cada x scrolls, pausa random
            if i % 10 == 0:
                minutos = random.uniform(3, 7)
                print(f'\n\n\nHaciendo pausa de {round(minutos)} minutos\n\n\n')
                time.sleep(minutos*60)


            # Cada x scrolls, scroll up to confuse bot detectors
            if i % 15 == 0:    
                self.driver.execute_script("""
                    var scrollt = document.querySelector('div[role="dialog"] ._aano')
                    scrollt.scrollTop = scrollt.scrollHeight - (scrollt.scrollHeight / 5)
                    """)

            # Print
            print(f'scrolleando {i} de {scrollear_inicial}')

            # Scroll down
            self.driver.execute_script("""
                var scrollt = document.querySelector('div[role="dialog"] ._aano')
                scrollt.scrollTop = scrollt.scrollHeight
                """)
            time.sleep(random.uniform(20, 25))


            # Nuevos seguidos
            # Español
            seguidos = self.driver.find_elements('xpath',f"(//div[@class='_aae-']/li)[position()>={len(seguidos_total)}]//*/a")
            seguidos = [u.text for u in seguidos if len(u.text)>0]
            if len(seguidos) == 0:
                #Ingles
                seguidos = self.driver.find_elements('xpath',f"//div[position()>={len(seguidos_total)}]//*/span/a[@role = 'link']")
                seguidos = [elem.get_attribute('href').split('/')[-2] for elem in seguidos]

            # agregamos los nuevos seguidos
            seguidos_total.extend(seguidos)
            print(f'Hasta el momento, se han encontrado {len(seguidos_total)} seguidos de {usuario}\n')

            # Si no encontro nada en las ultimas 4 iters, break
            cantidad_encontrados.append(len(seguidos_total))
            try:
                ultimos = 20
                if len(set(cantidad_encontrados[-ultimos:])) == 1 and len(cantidad_encontrados) > ultimos:
                    print('\n\n\nINTERRUPCION: NO SE ENCUENTRAN NUEVAS CUENTAS\n')
                    break
            except:
                pass

            # break si se alcanzo maximo de usuarios
            if len(seguidos_total) > cantidad_usuarios_a_scrapear:
                break

            time.sleep(random.uniform(20, 25))

        return seguidos_total

    def cantidad_usuarios(self, categoria):
        if categoria == 'seguidores':
            return self.can_seguidores
        else:
            return self.can_seguidos
        
    def logout(self):
        # Cerrar webdriver
        self.driver.close()
        print('\nDriver cerrado...\n')

In [6]:
seguidores_todos = {}
seguidos_todos = {}

start_total = time.time()

for usuario in usuarios_interes:
    start = time.time()
    
    porcentaje_encontrado = 0
    tiempo = 60*60
    
    try:
        while porcentaje_encontrado < 0.8:

            # Login
            sc = igScraper()
            sc.login(user, password, navegacion_con_headers = False)

            # Obtener seguidores/seguidos
            seguidos = sc.get_seguidos(usuario)

            porcentaje_encontrado = len(seguidos) / sc.cantidad_usuarios('seguidos')

            sc.logout()

            if porcentaje_encontrado < 0.8:
                tiempo *= 1.1
                print(f'\nHaciendo pausa de {round((tiempo/60/60),2)} horas.'), sleep(tiempo)

        # Guardarlo en dict
        seguidos_todos[usuario] = seguidos

        # Descarga
        sc.descargar(usuario, seguidos, 'seguidos')
        end = time.time()
        print(f'\n\n\n {usuario} tomo {round((end-start)/60)} minutos.\n\n\n')

        # pausa random
        horas = random.uniform(1, 2)
        print(f'\n\n\nHaciendo pausa inter usuario de {round(horas,2)} horas\n\n\n')
        time.sleep(horas*60*60)
        
    except:
        sc.logout()


end_total = time.time()

print(f'\n\n\nTodos los usuarios tomaron {round((end_total-start_total)/60)} minutos\n\n\n')

[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Driver [C:\Users\Felipe\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


 
Scrapeando seguidos de maggiesolariq




[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Driver [C:\Users\Felipe\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


scrolleando 1 de 8

Driver cerrado...

 
Scrapeando seguidos de lupetagliaferri


scrolleando 1 de 16
Hasta el momento, se han encontrado 24 seguidos de lupetagliaferri

scrolleando 2 de 16
Hasta el momento, se han encontrado 37 seguidos de lupetagliaferri

scrolleando 3 de 16
Hasta el momento, se han encontrado 49 seguidos de lupetagliaferri

scrolleando 4 de 16
Hasta el momento, se han encontrado 61 seguidos de lupetagliaferri

scrolleando 5 de 16
Hasta el momento, se han encontrado 73 seguidos de lupetagliaferri

scrolleando 6 de 16
Hasta el momento, se han encontrado 85 seguidos de lupetagliaferri

scrolleando 7 de 16
Hasta el momento, se han encontrado 97 seguidos de lupetagliaferri

scrolleando 8 de 16
Hasta el momento, se han encontrado 109 seguidos de lupetagliaferri

scrolleando 9 de 16
Hasta el momento, se han encontrado 121 seguidos de lupetagliaferri




Haciendo pausa de 5 minutos



scrolleando 10 de 16
Hasta el momento, se han encontrado 133 seguidos de lupetagliaferri



[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195


...

[WDM] - Driver [C:\Users\Felipe\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


Scrapeando seguidos de pyedlin


scrolleando 1 de 56
Hasta el momento, se han encontrado 24 seguidos de pyedlin

scrolleando 2 de 56
Hasta el momento, se han encontrado 37 seguidos de pyedlin

scrolleando 3 de 56
Hasta el momento, se han encontrado 49 seguidos de pyedlin

scrolleando 4 de 56
Hasta el momento, se han encontrado 61 seguidos de pyedlin

scrolleando 5 de 56
Hasta el momento, se han encontrado 73 seguidos de pyedlin

scrolleando 6 de 56
Hasta el momento, se han encontrado 85 seguidos de pyedlin

scrolleando 7 de 56
Hasta el momento, se han encontrado 97 seguidos de pyedlin

scrolleando 8 de 56
Hasta el momento, se han encontrado 109 seguidos de pyedlin

scrolleando 9 de 56
Hasta el momento, se han encontrado 121 seguidos de pyedlin




Haciendo pausa de 3 minutos



scrolleando 10 de 56
Hasta el momento, se han encontrado 133 seguidos de pyedlin

scrolleando 11 de 56
Hasta el momento, se han encontrado 145 seguidos de pyedlin

scrolleando 12 de 56
Hasta el momento, se han e

[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Driver [C:\Users\Felipe\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


 
Scrapeando seguidos de Victor_zimmermann


scrolleando 1 de 139
Hasta el momento, se han encontrado 12 seguidos de Victor_zimmermann

scrolleando 2 de 139
Hasta el momento, se han encontrado 25 seguidos de Victor_zimmermann

scrolleando 3 de 139
Hasta el momento, se han encontrado 35 seguidos de Victor_zimmermann

scrolleando 4 de 139
Hasta el momento, se han encontrado 47 seguidos de Victor_zimmermann

scrolleando 5 de 139
Hasta el momento, se han encontrado 59 seguidos de Victor_zimmermann

scrolleando 6 de 139
Hasta el momento, se han encontrado 70 seguidos de Victor_zimmermann

scrolleando 7 de 139
Hasta el momento, se han encontrado 82 seguidos de Victor_zimmermann

scrolleando 8 de 139
Hasta el momento, se han encontrado 94 seguidos de Victor_zimmermann

scrolleando 9 de 139
Hasta el momento, se han encontrado 106 seguidos de Victor_zimmermann




Haciendo pausa de 5 minutos



scrolleando 10 de 139
Hasta el momento, se han encontrado 118 seguidos de Victor_zimmermann

scrollea

scrolleando 86 de 139
Hasta el momento, se han encontrado 888 seguidos de Victor_zimmermann

scrolleando 87 de 139
Hasta el momento, se han encontrado 900 seguidos de Victor_zimmermann

scrolleando 88 de 139
Hasta el momento, se han encontrado 908 seguidos de Victor_zimmermann

scrolleando 89 de 139
Hasta el momento, se han encontrado 918 seguidos de Victor_zimmermann




Haciendo pausa de 4 minutos



scrolleando 90 de 139
Hasta el momento, se han encontrado 930 seguidos de Victor_zimmermann

scrolleando 91 de 139
Hasta el momento, se han encontrado 935 seguidos de Victor_zimmermann

scrolleando 92 de 139
Hasta el momento, se han encontrado 945 seguidos de Victor_zimmermann

scrolleando 93 de 139
Hasta el momento, se han encontrado 957 seguidos de Victor_zimmermann

scrolleando 94 de 139
Hasta el momento, se han encontrado 969 seguidos de Victor_zimmermann

scrolleando 95 de 139
Hasta el momento, se han encontrado 981 seguidos de Victor_zimmermann

scrolleando 96 de 139
Hasta el momen